# get_data

> async wrapper for asyncio requests


In [1]:
# | default_exp client.get_data


In [13]:
# | exporti
from typing import Optional, Union

from pprint import pprint

import httpx
import aiohttp

import domolibrary.client.DomoAuth as dmda
import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.DomoError as de


In [3]:
# | export
async def get_data_aiohttp(
    url: str,
    method: str,
    auth: dmda.DomoAuth,
    content_type: Optional[dict] = None,
    headers: Optional[dict] = None,
    # if no session passed by default will create and close session during execution
    session: Optional[aiohttp.ClientSession] = None,
    body: Union[dict, str, None] = None,
    params: Optional[dict] = None,
    debug_api: bool = False,
    process_stream: bool = False,
    stream_chunks: int = 10
) -> rgd.ResponseGetData:
    """async wrapper for asyncio requests"""

    if auth and not auth.token:
        await auth.get_auth_token()

    if headers is None:
        headers = {}

    is_close_session = False

    if session is None:
        is_close_session = True
        session = session or aiohttp.ClientSession()

    headers = {
        "Content-Type": content_type or "application/json",
        "Connection": "keep-alive",
        "accept": "application/json, text/plain",
        **headers,
    }

    if auth:
        headers.update(**auth.auth_header)

    if debug_api:
        pprint(
            {
                "method": method,
                "url": url,
                "headers": headers,
                "json": body,
                "params": params,
            }
        )

    try:
        if headers.get("Content-Type") == "application/json":
            res = await session.request(
                method=method.upper(),
                url=url,
                headers=headers,
                json=body,
                params=params,
            )

        elif body is not None:
            res = await session.request(
                method=method.upper(),
                url=url,
                headers=headers,
                data=body,
                params=params,
            )

        else:
            res = await session.request(
                method=method.upper(), url=url, headers=headers, params=params
            )
        return await rgd.ResponseGetData._from_aiohttp_response(res, auth=auth, process_stream=process_stream, stream_chunks=stream_chunks)

    except Exception as e:
        print(e)

    finally:
        if is_close_session:
            await session.close()


#### sample implementation of get_data

During execution `get_data()` will attempt to retrieve exchange credentials for an auth token using the `dmda.DomoFullAuth.get_auth_token()` method.

Then the appropriate headers will be passed to the request.


In [4]:
import os

full_auth = dmda.DomoFullAuth(
    domo_instance="domo-dojo",
    domo_username="jae@onyxreporting.com",
    domo_password=os.environ["DOJO_PASSWORD"],
)

await full_auth.get_auth_token()

url = "https://domo-dojo.domo.com/api/content/v2/users/me"

try:
    res = await get_data_aiohttp(url=url, method="get", auth=full_auth)

except Exception as e:
    print(e)

assert res.is_success

In [14]:
#|export 

class GetData_Error(de.DomoError):
    def __init__(self, message, url):
        super().__init__( message = message, domo_instance = url)
        

In [18]:
# | export
async def get_data(
    url: str,
    method: str,
    auth: dmda.DomoAuth,
    content_type: Optional[dict] = None,
    headers: Optional[dict] = None,
    body: Union[dict, str, None] = None,
    params: Optional[dict] = None,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
    return_raw: bool = False,
    is_follow_redirects: bool = False,
    timeout = 5
) -> rgd.ResponseGetData:
    """async wrapper for asyncio requests"""

    if debug_api:
        print("🐛 debugging get_data")

    if auth and not auth.token:
        await auth.get_auth_token()

    if headers is None:
        headers = {}

    headers = {
        "Content-Type": content_type or "application/json",
        "Connection": "keep-alive",
        "accept": "application/json, text/plain",
        **headers,
    }

    if auth:
        headers.update(**auth.auth_header)

    if debug_api:
        pprint(
            {
                "method": method,
                "url": url,
                "headers": headers,
                "body": body,
                "params": params,
            }
        )

    is_close_session = False if session else True

    session = session or httpx.AsyncClient()


    attempt = 1
    max_attempt = 4
    
    while attempt <= max_attempt:
        try:
            if isinstance(body, dict) or isinstance(body, list):
                if debug_api:
                    print("get_data: sending json")
                res = await getattr(session, method.lower())(
                    url=url,
                    headers=headers,
                    json=body,
                    params=params,
                    follow_redirects=is_follow_redirects,
                    timeout = timeout
                )

            elif body:
                if debug_api:
                    print("get_data: sending data")

                res = await getattr(session, method.lower())(
                    url=url,
                    headers=headers,
                    data=body,
                    params=params,
                    follow_redirects=is_follow_redirects,
                    timeout = timeout
                )

            else:
                if debug_api:
                    print("get_data: no body")

                res = await getattr(session, method.lower())(
                    url=url,
                    headers=headers,
                    params=params,
                    follow_redirects=is_follow_redirects,
                    timeout = timeout
                )

            if debug_api:
                print("get_data_response", res)

            if return_raw:
                return res

            return rgd.ResponseGetData._from_httpx_response(res, auth=auth)

        except Exception as e:
            print(f"ℹ️ get_data error - {e} at {url}")
            attempt +=1

            if attempt == max_attempt:
                raise GetData_Error(url=url, message=e)


        finally:
            if is_close_session:
                await session.aclose()



In [6]:
import os
import pandas as pd

auth = dmda.DomoTokenAuth(
    domo_instance="domo-dojo", domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

url = f"https://{ auth.domo_instance}.domo.com/api/content/v2/users"

res = await get_data(url=url, method="GET", auth=auth, debug_api=False)

# [prop for prop in res.__dict__.keys() if not prop.startswith('__')]

pd.DataFrame(res.response[0:5])

,id,invitorUserId,displayName,userName,emailAddress,accepted,userType,timeZone,modified,created,role,roleId,rights,active,pending,systemUser,anonymous
0,0,2,monitor,monitor,monitor@domo.com,False,DOMO_SUPPORT,Etc/GMT+7,1588877394091,1464820854,Privileged,1,31.0,True,True,True,True
1,1006847540,1893952720,Marc-Anton Clavel,marcanton.clavel@domo.com,marcanton.clavel@domo.com,False,USER,NaN,1665166064000,1618579073,Privileged,2,31.0,True,True,False,True
2,1012895591,1893952720,JeMiller,JeMiller@marketaxess.com,JeMiller@marketaxess.com,True,USER,NaN,1657051684429,1657049419,NaN,2097317660,NaN,True,False,False,False
3,1024352013,1893952720,Michael Shrifteylik,mshrifteylik@sportsresearch.com,mshrifteylik@sportsresearch.com,True,USER,NaN,1669658030028,1667838056,Privileged,2,31.0,True,False,False,False
4,1026684838,1893952720,Ghofrane Yahmadi,ghofrane.yahmadi@audibene.de,ghofrane.yahmadi@audibene.de,False,USER,NaN,1618579110000,1618579109,Privileged,2,31.0,True,True,False,True


# Looper


In [7]:
# | export
class LooperError(Exception):
    def __init__(self, loop_stage: str, message):

        super().__init__(f"{loop_stage} - {message}")

In [8]:
# | export
async def looper(
    auth: dmda.DomoAuth,
    session: httpx.AsyncClient,
    url,
    offset_params,
    arr_fn: callable,
    loop_until_end: bool = False,
    method="POST",
    body: dict = None,
    fixed_params: dict = None,
    offset_params_in_body: bool = False,
    body_fn=None,
    limit=1000,
    skip=0,
    maximum=2000,
    debug_api: bool = False,
    debug_loop: bool = False,
) -> rgd.ResponseGetData:

    maximum = maximum or 0

    is_close_session = False

    if not session:
        session = httpx.AsyncClient()
        is_close_session = True

    allRows = []
    isLoop = True

    res = None

    if maximum < limit and not loop_until_end:
        limit = maximum

    while isLoop:
        params = fixed_params or {}

        if offset_params_in_body:
            body[offset_params.get("offset")] = skip
            body[offset_params.get("limit")] = limit

        else:
            params[offset_params.get("offset")] = skip
            params[offset_params.get("limit")] = limit

        if body_fn:
            try:
                body = body_fn(skip, limit)

            except Exception as e:
                await session.aclose()
                raise LooperError(
                    loop_stage="processing body_fn", message=str(e)
                ) from e

        if debug_loop:
            print(f"\n🚀 Retrieving records {skip} through {skip + limit} via {url}")
            # pprint(params)

        res = await get_data(
            auth=auth,
            url=url,
            method=method,
            params=params,
            session=session,
            body=body,
            debug_api=debug_api,
        )

        if not res.is_success:
            if is_close_session:
                await session.aclose()
            return res

        try:
            newRecords = arr_fn(res)

        except Exception as e:
            await session.aclose()
            raise LooperError(loop_stage="processing arr_fn", message=str(e)) from e

        allRows += newRecords

        if loop_until_end and len(newRecords) != 0:
            maximum = maximum + limit

        if debug_loop:
            print({"all_rows": len(allRows), "new_records": len(newRecords)})

        if len(allRows) >= maximum or len(newRecords) == 0:
            if debug_loop:
                print(
                    f"\n🎉 Success - {len(allRows)} records retrieved from {url} in query looper\n"
                )

            break

        skip += len(newRecords)

        if skip + limit > maximum:
            limit = maximum - len(allRows)

        if debug_loop:
            print(f"skip: {skip}, limit: {limit}")

    if is_close_session:
        await session.aclose()

    return await rgd.ResponseGetData._from_looper(res=res, array=allRows)

#### sample implementation of httpx looper


In [9]:
# | hide
import os
import pandas as pd

session = httpx.AsyncClient()

sql = "SELECT * FROM TABLE"
dataset_id = os.environ["DOJO_DATASET_ID"]

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-dojo", domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

url = f"https://{token_auth.domo_instance}.domo.com/api/query/v1/execute/{dataset_id}"

offset_params = {
    "offset": "offset",
    "limit": "limit",
}

maximum = 10
skip = 0


def body_fn(skip, limit):
    return {"sql": f"{sql} limit {limit} offset {skip}"}


def arr_fn(res: rgd.ResponseGetData):
    rows_ls = res.response.get("rows")
    columns_ls = res.response.get("columns")
    output = []
    for row in rows_ls:
        new_row = {}
        for index, column in enumerate(columns_ls):
            new_row[column] = row[index]
        output.append(new_row)
    return output


res = await looper(
    auth=token_auth,
    method="POST",
    url=url,
    offset_params=offset_params,
    skip=skip,
    maximum=maximum,
    arr_fn=arr_fn,
    body_fn=body_fn,
    debug_api=False,
    debug_loop=False,
    loop_until_end=False,
    session=session,
)

await session.aclose()

pd.DataFrame(res.response[0:5])

,objectID,url,Title,article,views,created_dt,published_dt
0,000004785,https://domo-support.domo.com/s/article/360047...,Backing Up Workbench 4 Jobs,Important: Support for Workbench 4 ended on ...,138,2022-10-24T22:30:00,2022-10-24T22:42:00
1,000004807,https://domo-support.domo.com/s/article/360044...,Backing Up Workbench 5 Jobs,Backing up DataSet jobs is an often overlooked...,47,2022-10-24T22:31:00,2022-10-24T22:41:00
2,000004785,https://domo-support.domo.com/s/article/360047...,Backing Up Workbench 4 Jobs,Important: Support for Workbench 4 ended on ...,139,2022-10-24T22:30:00,2022-10-24T22:42:00
3,000004081,https://domo-support.domo.com/s/article/360043...,Beast Mode Functions Reference Guide,IntroYou can use this reference guide to learn...,826,2022-10-24T21:20:00,2022-10-24T22:40:00
4,000004508,https://domo-support.domo.com/s/article/360043...,Fun Sample DataSets,IntroIt's hard learning how to perform advance...,365,2022-10-24T22:13:00,2022-10-24T22:39:00


## Aiohttp Looper DEPRECATED


In [10]:
# | export
async def looper_aiohttp(
    auth: dmda.DomoAuth,
    session: aiohttp.ClientSession,
    url,
    offset_params,
    arr_fn: callable,
    loop_until_end: bool = False,
    method="POST",
    body: dict = None,
    fixed_params: dict = None,
    offset_params_in_body: bool = False,
    body_fn=None,
    limit=1000,
    skip=0,
    maximum=2000,
    debug_api: bool = False,
    debug_loop: bool = False,
) -> rgd.ResponseGetData:

    is_close_session = False

    if not session:
        session = aiohttp.ClientSession()
        is_close_session = True

    allRows = []
    isLoop = True

    res = None

    if maximum < limit:
        limit = maximum

    while isLoop:
        params = fixed_params or {}

        if offset_params_in_body:
            body[offset_params.get("offset")] = skip
            body[offset_params.get("limit")] = limit

        else:
            params[offset_params.get("offset")] = skip
            params[offset_params.get("limit")] = limit

        if body_fn:
            try:
                body = body_fn(skip, limit)

            except Exception as e:
                await session.aclose()
                raise LooperError(
                    loop_stage="processing body_fn", message=str(e)
                ) from e

        if debug_loop:
            print(f"\n🚀 Retrieving records {skip} through {skip + limit} via {url}")
            # pprint(params)

        res = await get_data_aiohttp(
            auth=auth,
            url=url,
            method=method,
            params=params,
            session=session,
            body=body,
            debug_api=debug_api,
        )

        if not res.is_success:
            if is_close_session:
                await session.aclose()
            return res

        try:
            newRecords = arr_fn(res)

        except Exception as e:
            await session.close()
            raise LooperError(loop_stage="processing arr_fn", message=str(e)) from e

        allRows += newRecords

        if loop_until_end and len(newRecords) != 0:
            maximum = maximum + limit

        if debug_loop:
            print({"all_rows": len(allRows), "new_records": len(newRecords)})

        if len(allRows) >= maximum or len(newRecords) == 0:
            if debug_loop:
                print(
                    f"\n🎉 Success - {len(allRows)} records retrieved from {url} in query looper\n"
                )

            break

        skip += len(newRecords)

        if skip + limit > maximum:
            limit = maximum - len(allRows)

            if debug_loop:
                print(f"skip: {skip}, limit: {limit}")

    if is_close_session:
        await session.close()

    return await rgd.ResponseGetData._from_looper(res=res, array=allRows)

In [11]:
# | hide
import os
import pandas as pd

session = aiohttp.ClientSession()

sql = "SELECT * FROM TABLE"
dataset_id = os.environ["DOJO_DATASET_ID"]

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-dojo", domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

url = f"https://{token_auth.domo_instance}.domo.com/api/query/v1/execute/{dataset_id}"

offset_params = {
    "offset": "offset",
    "limit": "limit",
}

maximum = 10
skip = 0


def body_fn(skip, limit):
    return {"sql": f"{sql} limit {limit} offset {skip}"}


def arr_fn(res: rgd.ResponseGetData):
    rows_ls = res.response.get("rows")
    columns_ls = res.response.get("columns")
    output = []
    for row in rows_ls:
        new_row = {}
        for index, column in enumerate(columns_ls):
            new_row[column] = row[index]
        output.append(new_row)
    return output


res = await looper_aiohttp(
    auth=token_auth,
    method="POST",
    url=url,
    offset_params=offset_params,
    skip=skip,
    maximum=maximum,
    arr_fn=arr_fn,
    body_fn=body_fn,
    debug_api=False,
    debug_loop=False,
    loop_until_end=False,
    session=session,
)

await session.close()

pd.DataFrame(res.response[0:5])

,objectID,url,Title,article,views,created_dt,published_dt
0,000004785,https://domo-support.domo.com/s/article/360047...,Backing Up Workbench 4 Jobs,Important: Support for Workbench 4 ended on ...,138,2022-10-24T22:30:00,2022-10-24T22:42:00
1,000004807,https://domo-support.domo.com/s/article/360044...,Backing Up Workbench 5 Jobs,Backing up DataSet jobs is an often overlooked...,47,2022-10-24T22:31:00,2022-10-24T22:41:00
2,000004785,https://domo-support.domo.com/s/article/360047...,Backing Up Workbench 4 Jobs,Important: Support for Workbench 4 ended on ...,139,2022-10-24T22:30:00,2022-10-24T22:42:00
3,000004081,https://domo-support.domo.com/s/article/360043...,Beast Mode Functions Reference Guide,IntroYou can use this reference guide to learn...,826,2022-10-24T21:20:00,2022-10-24T22:40:00
4,000004508,https://domo-support.domo.com/s/article/360043...,Fun Sample DataSets,IntroIt's hard learning how to perform advance...,365,2022-10-24T22:13:00,2022-10-24T22:39:00


In [12]:
# | hide
import nbdev

nbdev.nbdev_export()